In [1]:
try:
    import spacy
except:
    import spacy
import json
from datetime import datetime
name = 'finance_course1'
JSON_path = '/home/ubuntu/Questions_generation/Financial Markets Course 2.json'
sections_num_max= 'all'
%pdb on


/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Automatic pdb calling has been turned ON


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
#questions
fig = mcq.plot_similarity_matrix((questions_df[questions_used]).to_list())
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000)
fig.show()
#both
fig = mcq.plot_similarity_matrix((questions_df[questions_used]+' '+questions_df['selected_ans']).to_list())
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000)

In [2]:
from MCQ import flanT5MCQ
from datetime import datetime
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
dir_path = '../outputs/'+datetime.now().strftime("%d_%m_%y")+'_'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open(JSON_path) as f:
  result = json.load(f)

sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
if sections_num_max!='all':
  sections = sections[:sections_num_max]
print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]
min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
if 'mcq' in locals():
  del mcq
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)
'''
cpu_model = mcq.model.to('cpu')
del mcq.model
mcq.model = cpu_model
mcq.push_model_to_GPU(mcq.model)
'''
with torch.no_grad():
  questions_df = mcq.generate_questions(sections=sections,org_sections=org_text,sections_ranks=np.ones(len(sections)))
questions_df.to_pickle(dir_path+'GQ.pickle')
print(f"Stage 1:{dir_path+'GQ.pickle'} has been saved")
torch.cuda.empty_cache()
if mcq.COMPUTE_ANSWERS:
  mcq.push_model_to_GPU(mcq.QA.model)
  questions_df = mcq.QA.select_best_answer(questions_df)
  questions_df.to_pickle(dir_path+'QG+QA.pickle')
  print(f"Stage 2:{dir_path+'QG+QA.pickle'} has been saved")
  questions_df = mcq.QG.create_question_MixQG(questions_df)
  questions_df.to_pickle(dir_path+'GQ+QA+GQ.pickle')
  print(f"Stage 3:{dir_path+'GQ+QA+GQ.pickle'} has been saved")
  questions_used = 'new_question'
  questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
  questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
  q_sim_mat,q_ans_sim_mat = mcq.find_similarity(questions_df[questions_used].to_list(),
                                            answers = (questions_df[questions_used]+' '+questions_df['selected_ans']).to_list())
  filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                  q_sim_mat = q_sim_mat, ans_sim_mat = q_ans_sim_mat,
                                  n_thrs=20, return_index=True)
  questions_df['use_question']=False
  questions_df.loc[filter_idx,'use_question']=True
  #questions_df = questions_df.iloc[filter_idx,:]
with open(f'{dir_path}questions_full.txt', 'w') as f:
    qs_text =[mcq.show_qs(questions_df,i) for i in questions_df.index.values]
    text_outuput=''
    for i,qs in zip(questions_df.index.values,qs_text):
        text_outuput += f"({i})TAKEN?{questions_df['use_question'][i]} RQUGE:{round(questions_df['RQUGE'][i],4)}"+'\n'+ qs+"\n\n"
    f.write(text_outuput)
    print(f"Stage 4:{dir_path}questions_full.txt has been saved")
    
with open(f'{dir_path}questions.txt', 'w') as f:
    text_outuput=''
    for i in np.unique(questions_df.section_n_chunk):
      qs_in_sections = questions_df.query(f'section_n_chunk == {i} and use_question == True')
      for q,a in zip(qs_in_sections[questions_used],qs_in_sections.selected_ans):
        text_outuput+=f'Q:{q}\nA:{a}\n'
      text_outuput+= '-'*50 + '\n'
    f.write(text_outuput)
    print(f"Stage 5:{dir_path}questions.txt has been saved")

Limit questions to be from the all first sections


Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


Abbreviation 	 Definition
{}


0it [00:00, ?it/s]

Total text shape is torch.Size([1, 376])
Qs:Who were some of these institutions that collapsed?
Qs:The speaker is going to talk about what in the context?
Qs:What happened after the stock market collapsed
Qs:How did this lecture begin?
Qs:When the stock market collapsed around the world, what happened?
Qs:Why was there a pre-break around 2000?
Qs:What is the speaker going to talk about?


...

Stage 1:../outputs/03_01_23_finance_course1/GQ.pickle has been saved


Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


Stage 2:../outputs/03_01_23_finance_course1/QG+QA.pickle has been saved


100%|██████████| 60/60 [01:26<00:00,  1.44s/it]


Stage 3:../outputs/03_01_23_finance_course1/GQ+QA+GQ.pickle has been saved
Stage 4:../outputs/03_01_23_finance_course1/questions_full.txt has been saved
Stage 5:../outputs/03_01_23_finance_course1/questions.txt has been saved


In [2]:
!pip install transformers
!pip install -U sentencepiece 
!pip install sentence_transformers
!pip install scispacy
!pip install -U https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
import nltk
nltk.download('punkt')


  Using cached scispacy-0.5.1-py3-none-any.whl (44 kB)
  Using cached conllu-4.5.2-py2.py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 75.3 MB/s eta 0:00:0000:0100:01
  Using cached pysbd-0.3.4-py3-none-any.whl (71 kB)
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz (15.9 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870856 sha256=e340369d04126ffde1ac6e74861fd5179807f56201403f3db49e00f6d1dcb424
  Stored in directory: /home/ubuntu/.cache/pip/wheels/f5/2e/39/9c9d425a1d34c06409420f7c65c5e10a56f7b149a3c37cdfa6
Successfully built en-core-sci-sm


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
try:import spacy
except: import spacy
%pdb on
import torch
torch.manual_seed(10)
#!python -m spacy download en_core_web_lg
try:
  from compute_question_and_answer import compute_question_and_answer
except:
  from compute_question_and_answer import compute_question_and_answer
import json
name = 'deep_learning2'
sections_num_max = 'all'
JSON_path = '/home/ubuntu/Questions_generation/Deep learning.json'
with open(JSON_path) as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
if sections_num_max!='all':
  sections = sections[:sections_num_max]
  print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

out = compute_question_and_answer(sections,org_text,save_name=name)


/home/ubuntu/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Automatic pdb calling has been turned ON


Token indices sequence length is longer than the specified maximum sequence length for this model (768 > 512). Running this sequence through the model will result in indexing errors


Abbreviation 	 Definition
{'SGD': 'Stochastic Gradient Descent', 'ReLU': 'Rectified Linear Unit', 'RGB': 'Red, Green, Blue', 'CIFAR': 'Canadian Institute for Advanced Research', 'GPUs': 'Graphics Processing Units', 'RNN': 'Recurrent Neural Network', 'CNN': 'Convolution Neural Network', 'RNNs': 'Recurrent Neural Networks', 'LSTM': 'Long Short-term Memory'}


0it [00:00, ?it/s]

...

Stage 1:../outputs/16_01_23_deep_learning2/GQ.pickle has been saved


Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


Stage 2:../outputs/16_01_23_deep_learning2/QG+QA.pickle has been saved


100%|██████████| 48/48 [01:10<00:00,  1.48s/it]


Stage 3:../outputs/16_01_23_deep_learning2/GQ+QA+GQ.pickle has been saved
Stage 4:../outputs/16_01_23_deep_learning2/deep_learning2_questions_full.txt has been saved
Stage 5:../outputs/16_01_23_deep_learning2/deep_learning2_questions.txt has been saved
--------------------------------------------------
distractors not found! What do machine-learning systems increasingly use?
--------------------------------------------------
not 
Which of these is not a type of machine learning? Which of these is  a type of machine learning?
Which of these is not a type of machine learning? --- Human engineering
 ['Pattern recognition', 'Deep neural networks', 'Match news items, posts or products with users’ interests.']
--------------------------------------------------
distractors not found! What are some of the applications of machine learning?
--------------------------------------------------
distractors not found! What procedure do most practitioners use?
-----------------------------------------

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("/path/to/s2v_reddit_2015_md")

doc = nlp("A sentence about natural language processing.")
assert doc[3:6].text == "natural language processing"
freq = doc[3:6]._.s2v_freq
vector = doc[3:6]._.s2v_vec
most_similar = doc[3:6]._.s2v_most_similar(3)

In [ ]:
def model_loader(kind, name, load):
  name = kind + ":" + name
  if name not in pre_loaded:
    pre_loaded[name] = load()
  return pre_loaded[name]

pre_loaded = {}
s2v = model_loader(
    'sense2vec',
    "s2v",
    lambda : Sense2Vec().from_disk('./support/models/sense2vec/s2v_reddit_2019_lg') # large 
)

def find_distractors_s2v(term, n=5):
    try:
        candidates = [
                    result[0].split("|")[0].replace("_"," ")
                    for result in s2v.most_similar(term, n=30)
                    if result[0].split("|")[1] == term.split("|")[1]
                    and result[0] == s2v.get_best_sense(result[0].split("|")[0].replace("_"," "))
        ]
        # print(candidates)
        
        distractors = filter_distractors(term.split("|")[0].replace("_"," "), candidates, n=n ,threshold = 0.7)

        if distractors:
            return distractors
    except ValueError as e:
        return

In [9]:
import pandas as pd
questions_df = pd.read_pickle('/home/ubuntu/Questions_generation/outputs/16_01_23_deep_learning/GQ+QA+GQ.pickle')
import numpy as np

output = []
for i in range(max(questions_df.section_n)+1):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():
                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])
                section_output['score'].append(q['RQUGE'])
                section_output['details'].append(q['details'])
                section_output['context'].append(q['text'])
    output.append(section_output)

KeyError: 'RQUGE'

> /home/ubuntu/miniforge3/lib/python3.9/site-packages/pandas/core/indexes/base.py(3805)get_loc()
   3803                 return self._engine.get_loc(casted_key)
   3804             except KeyError as err:
-> 3805                 raise KeyError(key) from err
   3806             except TypeError:
   3807                 # If we have a listlike key, _check_indexing_error will raise



In [10]:
questions_df

,section_n,section_n_chunk,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,details,new_question
0,0,0.0,1.0,Machine-learning systems are used to identify ...,"Increasingly, these applications make use what?",0.000630,Deep neural networks,Class of techniques called deep learning.,"Increasingly, these applications make use of a...",A class of techniques called deep learning.,Depth,Deep learning.,Deep learning,,class of techniques called deep learning.,Class of techniques called deep learning.,,What do machine-learning systems increasingly ...
1,0,0.0,1.0,Machine-learning systems are used to identify ...,Which of these is not a type of machine learning?,0.008745,Human engineering.,Human engineering,Deep neural networks,"Identify objects in images, transcribe speech ...",Human engineering,Humans,Deep neural networks,Domain expertise,human engineering,Human engineering,NEGATIVE: not,Which of these is not a type of machine learning?
2,0,0.0,1.0,Machine-learning systems are used to identify ...,Machine-learning systems are used to identify ...,0.016903,Machine-learning systems are used to identify ...,Machine-learning systems are used to identify ...,Transcribing speech into text requires careful...,The key aspect of deep learning is that these ...,Search,Text,Transscribe speech into text,Translate speech into text,machine-learning systems are used to identify ...,Machine-learning systems are used to identify ...,,What are some of the applications of machine l...
3,0,0.0,1.0,Machine-learning systems are used to identify ...,How does deep learning differ from conventiona...,0.065550,This is a list of machine-learning application...,Machine-learning systems can process natural d...,These layers of features are not designed by h...,The key aspect of deep learning is that these ...,They are learned from data using a general-pur...,Requires very little engineering by hand,Required careful engineering and considerable ...,They are learned from data using a general-pur...,these layers of features are not designed by h...,These layers of features are not designed by h...,,What is the key aspect of deep learning?
4,0,0.0,1.0,Machine-learning systems are used to identify ...,What is the key aspect of deep learning?,0.077957,The key aspect of deep learning is that these ...,These layers of features are not designed by h...,It has turned out to be very good at discoveri...,Deep learning is making major advances in solv...,No engineering by hand,Requires very little engineering by hand,They are learned from data using a general-pur...,These layers of features are not designed by h...,these layers of features are not designed by h...,These layers of features are not designed by h...,,What is the key aspect of deep learning?


In [17]:
import spacy
from negspacy.negation import Negex

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

doc = nlp("She does not like Steve Jobs but likes Apple products.")
for e in doc.ents:
    print(e.text, e._.negex)

Steve Jobs True
Apple False


In [20]:
{"am not":"am","is not":"is","are not":"are","was not":"was","were not":"were","haven't been":"have been","hasn't been":"has been","hadn't been":"had been","will not be":"will be","would not be":"would be","should not be":"should be","could not be":"could be","must not be":"must be","might not be":"might be","don't":"do","does not":"does","did not":"did","has not":"has","have not":"have","had not":"had","can not":"can","could not":"could","shall not":"shall","should not":"should","will not":"will","would not":"would","may not":"may","might not":"might","must not":"must","ought not":"ought","need not":"need","dare not":"dare","used not":"used","needn't":"need","daren't":"dare","mustn't":"must","shouldn't":"should","wouldn't":"would","couldn't":"could","wasn't":"was","weren't":"were","isn't":"is","aren't":"are","didn't":"did","doesn't":"does","hasn't":"has","haven't":"have","hadn't":"had","can't":"can","we're not":"we're","they're not":"they're","you're not":"you're","i'm not":"i'm","he's not":"he's","she's not":"she's","it's not":"it's","we're not":"we are","they're not":"they are","you're not":"you are","i'm not":"i am","he's not":"he is","she's not":"she is","it's not":"it is","we are not":"we're","they are not":"they're","you are not":"you're","i am not":"i'm"
                          ,"he is not":"he's","she is not":"she's","it is not":"it's","we are not": "we're",
    "they are not": "they're",
    "you are not": "you're",
    "i am not": "i'm",
    "he is not": "he's",
    "she is not": "she's",
    "it is not": "it's",
    "am not": "am",
    "is not": "is",
    "are not": "are",
    "was not": "was",
    "were not": "were",
    "haven't been": "have been",
    "hasn't been": "has been",
    "hadn't been": "had been",
    "will not be": "will be",
    "would not be": "would be",
    "should not be": "should be",
    "could not be": "could be",
    "must not be": "must be",
    "might not be": "might be",
    "don't": "do",
    "does not": "does",
    "did not": "did",
    "has not": "has",
    "have not": "have",
    "had not": "had",
    "can not": "can",
    "could not": "could",
    "shall not": "shall",
    "should not": "should",
    "will not": "will",
    "would not": "would",
    "may not": "may",
    "might not": "might",
    "must not": "must",
    "ought not": "ought",
    "need not": "need",
    "dare not": "dare",
    "used not": "used",
    "needn't": "need",
    "daren't": "dare",
    "mustn't": "must",
    "shouldn't": "should",
    "wouldn't": "would",
    "couldn't": "could",
    "wasn't": "was",
    "weren't": "were",
    "isn't": "is",
    "aren't": "are",
    "didn't": "did",
    "doesn't": "does",
    "hasn't": "has",
    "haven't": "have",
    "hadn't": "had",
    "can't": "can",
    "we're not": "we're",
    "they're not": "they're",
    "you're not": "you're",
    "i'm not": "i'm",
    "he's not": "he's",
    "she's not": "she's",
    "it's not": "it's",
    "am not": "am",
    "is not": "is",
    "are not": "are",
    "was not": "was",
    "were not": "were",
    "haven't been": "have been",
    "hasn't been": "has been",
    "hadn't been": "had been",
    "will not be": "will be",
    "would not be": "would be",
    "should not be": "should be",
    "could not be": "could be",
    "must not be": "must be",
    "might not be": "might be",
    "don't": "do",
    "does not": "does",
    "did not": "did",
    "has not": "has",
    "have not": "have",
    "had not": "had",
    "can not": "can",
    "could not": "could",
    "shall not": "shall",
    "should not": "should",
    "will not": "will",
    "would not": "would",
    "may not": "may",
    "might not": "might",
    "must not": "must",
    "ought not": "ought",
    "need not": "need",
    "dare not": "dare",
    "used not": "used",
    "needn't": "need",
    "daren't": "dare",
    "mustn't": "must",
    "shouldn't": "should",
    "wouldn't": "would",
    "couldn't": "could",
    "wasn't": "was",
    "weren't": "were",
    "isn't": "is",
    "aren't": "are",
    "didn't": "did",
    "doesn't": "does",
    "hasn't": "has",
    "haven't": "have",
    "hadn't": "had",
    "can't": "can"
}


{'am not': 'am',
 'is not': 'is',
 'are not': 'are',
 'was not': 'was',
 'were not': 'were',
 "haven't been": 'have been',
 "hasn't been": 'has been',
 "hadn't been": 'had been',
 'will not be': 'will be',
 'would not be': 'would be',
 'should not be': 'should be',
 'could not be': 'could be',
 'must not be': 'must be',
 'might not be': 'might be',
 "don't": 'do',
 'does not': 'does',
 'did not': 'did',
 'has not': 'has',
 'have not': 'have',
 'had not': 'had',
 'can not': 'can',
 'could not': 'could',
 'shall not': 'shall',
 'should not': 'should',
 'will not': 'will',
 'would not': 'would',
 'may not': 'may',
 'might not': 'might',
 'must not': 'must',
 'ought not': 'ought',
 'need not': 'need',
 'dare not': 'dare',
 'used not': 'used',
 "needn't": 'need',
 "daren't": 'dare',
 "mustn't": 'must',
 "shouldn't": 'should',
 "wouldn't": 'would',
 "couldn't": 'could',
 "wasn't": 'was',
 "weren't": 'were',
 "isn't": 'is',
 "aren't": 'are',
 "didn't": 'did',
 "doesn't": 'does',
 "hasn't": 'h

In [3]:
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk("/home/ubuntu/Questions_generation/s2v_old")

nlp = spacy.load("en_core_web_sm")
s2v = nlp.add_pipe("sense2vec")
#s2v.from_disk("/path/to/s2v_reddit_2015_md")

doc = nlp("A sentence about Apple.")
assert doc[3].text == "Apple"
freq = doc[3]._.s2v_freq
vector = doc[3]._.s2v_vec
most_similar = doc[3]._.s2v_most_similar(3)

ValueError: Can't find key Apple|PROPN in table

> /home/ubuntu/miniforge3/lib/python3.9/site-packages/sense2vec/sense2vec.py(209)most_similar()
    207         for key in keys:
    208             if key not in self:
--> 209                 raise ValueError(f"Can't find key {key} in table")
    210         if self.cache and self.cache["indices"].shape[1] >= n:
    211             n = min(len(self.vectors), n)



In [29]:
ui=['a) Inference for factors: rome, the final answer is yes ', 
    '(2) Inference for factors: rome, the final answer is (p)',
    'Inference for factors: rome is a method for editing the model of a transformer language.', 'Model editing of factual associations in transformer language models.', 'Model editing of factual associations in transformer language models', 'Rome is a method for modifying the weights of mid-layer feed-forward modules in a transformer-language model.']
output_a =[]
import re
for a in ui:
          cur_a = a.strip()
          cur_a = re.sub(r"^\([a-zA-Z]\)|^[a-zA-Z]\)", "", cur_a)
          cur_a = re.sub(r'[Tt]he final answer(:| is: |s are:| is) ?\([A-Za-z]\)\.?$','',cur_a, re.IGNORECASE)
          output_a.append(cur_a.strip().capitalize())
          
output_a

['Inference for factors: rome, the final answer is yes',
 '(2) inference for factors: rome,',
 'Inference for factors: rome is a method for editing the model of a transformer language.',
 'Model editing of factual associations in transformer language models.',
 'Model editing of factual associations in transformer language models',
 'Rome is a method for modifying the weights of mid-layer feed-forward modules in a transformer-language model.']

In [15]:

string = "I want to go home, yes. Do you want to come with me, yes"

# Use the sub() function to replace "yes" at the end of a sentence with "no"
modified_string = re.sub(r"\byes\.?$", "no", string, flags=re.IGNORECASE)

print(modified_string)

I want to go home, yes. Do you want to come with me, no


In [16]:
output = []
for i in np.unique(questions_df.section_n):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():
            if q['take']:
                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])
                section_output['score'].append(q['RQUGE'])
    output.append(section_output)

In [31]:
sections
dp = [len(x.split())>350 for x in sections]
print(dp)
len(list(filter(lambda x: x[1] is not None and dp[x[0]], enumerate(sections))))

[False, True]


1

In [11]:

text = ['Q:What was the derived uncertainty of the model?',
'A:4.1 minutes',
'Q:What is the uncertainty of the GBT model?',
'A:Gbt is the best performing model. the mean absolute error values in the table suggest that the performance is similar across all three algorithms. the derived uncertainty of the model was 4.1 minutes.',
'Q:What type of model is Barket-FM-DOSM?',
'A:Duration of surgery model',
'Q:What did the comparison lead to?',
'A:This comparison led to the conclusion that neither machine learning algorithms nor the dataset are the source of differences in the models performance. the major effector is that the set of features is the source.',
'Q:What is the main effector of differences in the model performance?',
'A:Dr. barket developed the model. the model is based on the surgery dataset. the main effector of such differences is the set of features.',
'Q:What is the most important feature?',
'A:Duration of surgery is the most important feature.',
'Q:3 out of 8 features selected are the same for which two methods?',
'A:Shap and pearson correlation.',
'Q:Where is Table 1 located?',
'A:Using machine learning techniques to develop supervised models that predict duration of surgery from features related to patients, physicians, and surgeries.',
'Q:What can be done with the predictions?',
'A:Use the dos value predicted by our model for surgery scheduling can decrease patient waiting time and maximize surgical staff idle time.']

%timeit mcq.sentence_model.encode(text)
%timeit [mcq.sentence_model.encode(t) for t in text]

10.7 ms ± 281 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
121 ms ± 894 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
import numpy as np
from collections import Counter
def check(args,prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    print(input_ids.shape)
    res = model.generate(input_ids, **args)
    return res['sequences'].shape
    output = tokenizer.batch_decode(res['sequences'], skip_special_tokens=True)

    output = [item.split("<sep>") for item in output]
    if all([len(cur_sen)==1 for cur_sen in output]):  
        output = [cur_sen[0] for cur_sen in output]
    if len(output)==1 and isinstance(output[0],list):
        output = output[0]
    output = [qs.strip() for qs in output]
    output = [*Counter(output)] #remove exactly similar questions if exist

    # Data originally sorted by probabilities, multiply be the penalities activated (`length_penalty` and/or `diversity_penalty`).
    # We want it to be sorted by the perplexity (PPL) score, the coherence of the sentence - probability normalized by the length of the sentence. 
    ppl = [np.exp(np.array(log_likelihoods.cpu()).mean() / np.array(len(cur_output.split())).mean()) for log_likelihoods,cur_output in zip(res['sequences_scores'],output)]
    sorted_idx  = np.argsort(ppl)
    return [output[id] for id in sorted_idx], [ppl[id] for id in sorted_idx]
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}
from transformers import T5Tokenizer,T5ForConditionalGeneration
checkpoint = "google/flan-t5-large"#"IsaacBot/flan-t5-small-mfaq-finetuned-question-generation-context-only"# "google/flan-t5-xl"#"google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
COMES_sections = ['COMET is a recently proposed trainable neuralbased evaluation metric developed to assess the quality of Machine Translation systems. In this paper, we explore the usage of COMET for evaluating Text Summarization systems – despite being trained on multilingual MT outputs, it performs remarkably well in monolingual settings, when predicting summarization output quality. We introduce a variant of the model – COMES – trained on the annotated summarization outputs that uses MT data for pre-training. We examine its performance on several datasets with human judgments collected for different notions of summary quality, covering several domains and languages.',
                  ' Since manual annotation for any generative task is costly and time consuming, automatic metrics are commonly used to measure progress during training and compare output from independent systems. Metrics Shared Task collocated with the WMT workshop since 2008 (Callison-Burch et al., 2008) advances in the MT models performance are accompanied by a continuous development of new automatic metrics. They are robust to both domain shifts and changes in annotation style. One of the issues making research on summary evaluation metrics difficult is lack of standardized framework for collecting human judgments.  We propose a variant of the model – COMES2 – that uses the annotated MT data for pre-training and is capable of predicting several aspects of summary quality. We evaluate our approach (Section 4) on selected datasets with various annotated styles. We examine the applicability of the COMET metric by Rei et al. (2020) that is trained on the data and capable of directly regressing a quality score.',
                  ' For a comprehensive survey on the summary evaluation resources see Koto et al. and system output (Papineni et al., 2002; Lin, 2004). Over the years, a variety of metrics were proposed for this task – based on question answering, similarity between summary and reference embeddings.',
                  ' COMET is a trained metric that, based on semantic similarities between the translated and reference texts, learns to output a score that resembles the human perception of translation quality. COMET uses a pre-trained multilingual language model to extract representations for each of the input sequences, which are then pooled and concatenated, before being processed with a stack of feed-forward layers that outputs a single numerical value.']
print(check(generator_args,['generate question:'+i for i in COMES_sections]))

torch.Size([1, 5])


/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


torch.Size([8, 13])


In [42]:
import re 
def filter_questions_by_re(q):

    # excluded expressions
    #start with
    who_start_re = r'([Ww]ho|WHO .*)'
    # What is the final answer to the question
    q_ref_re = r'[Ww]hat is|are the final answer|answers to the question|questions'
    starts_excluded_re = [who_start_re,q_ref_re]
    # must have expressions
    q_mark_re = r'.*\?'
    start_filter = [re.match(cur_re, q)!=None for cur_re in starts_excluded_re]
    print(q,re.search(q_mark_re, q))
    #return True if question is discard
    return any(start_filter) or re.search(q_mark_re, q)==None

print(filter_questions_by_re("Who is there?"))  # True
print(filter_questions_by_re("who is there?"))  # True
print(filter_questions_by_re("WHO is there?"))  # True

print(filter_questions_by_re("Where is he?"))   # False
print(filter_questions_by_re("Where is he"))   # False
print(filter_questions_by_re("Where is he ? "))   # False
print(filter_questions_by_re("Where is he  "))   # False
print(filter_questions_by_re("What is the final answer to the question ? "))   # False

print(filter_questions_by_re("Who are you?"))   # True
print(filter_questions_by_re("who are you?"))   # True
print(filter_questions_by_re("WHO are you?"))   # True
print(filter_questions_by_re("WHO are you"))   # True

Who is there? <re.Match object; span=(0, 13), match='Who is there?'>
True
who is there? <re.Match object; span=(0, 13), match='who is there?'>
True
WHO is there? <re.Match object; span=(0, 13), match='WHO is there?'>
True
Where is he? <re.Match object; span=(0, 12), match='Where is he?'>
False
Where is he None
True
Where is he ?  <re.Match object; span=(0, 13), match='Where is he ?'>
False
Where is he   None
True
What is the final answer to the question ?  <re.Match object; span=(0, 42), match='What is the final answer to the question ?'>
True
Who are you? <re.Match object; span=(0, 12), match='Who are you?'>
True
who are you? <re.Match object; span=(0, 12), match='who are you?'>
True
WHO are you? <re.Match object; span=(0, 12), match='WHO are you?'>
True
WHO are you None
True


In [26]:
import re
def check_start(text):
    regex = r'([Ww]ho|WHO .*)'
    match = re.match(regex, text)
    if match:
        return True
    return False

print(check_start("Who is there?"))  # True
print(check_start("who is there?"))  # True
print(check_start("WHO is there?"))  # True
print(check_start("Where is he?"))   # False
print(check_start("Who are you?"))   # True
print(check_start("who are you?"))   # True
print(check_start("WHO are you?"))   # True

True
True
True
False
True
True
True


In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

# Text to extract triplets from
text = 'Punta Cana is a resort town in the municipality of Higüey, in La Altagracia Province, the easternmost province of the Dominican Republic.'
name = 'finance_course2'
import json
JSON_path = '/home/ubuntu/Questions_generation/Financial Markets Course 2.json'
with open(JSON_path) as f:
  result = json.load(f)

text = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]

with open('/home/ubuntu/Questions_generation/outputs/03_01_23_finance_course2/questions.txt') as f:
    text = f.read()
text = text.replace('\nQ:','.\n ').replace('\nA:',' ').replace ('--------------------------------------------------','').split('.')
text = [' ']#text[:2]
# Tokenizer text
model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

# Generate
generated_tokens = model.generate(
    model_inputs["input_ids"].to(model.device),
    attention_mask=model_inputs["attention_mask"].to(model.device),
    **gen_kwargs,
)

# Extract text
decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

# Extract triplets
print(text)
for idx, sentence in enumerate(decoded_preds):
    print(f'Prediction triplets sentence {idx}')
    print(extract_triplets(sentence))

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


[' ']
Prediction triplets sentence 0
[{'head': 'John F. Kennedy School of Government', 'type': 'part of', 'tail': 'Harvard University'}]
Prediction triplets sentence 1
[{'head': 'World War II', 'type': 'has effect', 'tail': 'Holocaust'}]
Prediction triplets sentence 2
[{'head': 'World War I', 'type': 'followed by', 'tail': 'World War II'}, {'head': 'World War II', 'type': 'follows', 'tail': 'World War I'}]


In [8]:
try: import spacy   
except: import spacy

with open('/home/ubuntu/Questions_generation/outputs/03_01_23_finance_course2/questions.txt') as f:
    text = f.read()
text = text.replace('\nQ:','.\n ').replace('\nA:',' ').replace ('--------------------------------------------------','').split('.')
text = "what year Israel was established? 1948"
nlp = spacy.load("en_core_web_sm")
if isinstance(text,list):
    doc= nlp("\n".join(text))
else:
    doc =nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

from spacy import displacy
print(text)
def only_ents(text,doc):
    questions_mark_idx = text.find('?')

#displacy.serve(doc, style="ent")
displacy.render(doc, style='dep')

Israel 10 16 GPE
1948 34 38 DATE
what year Israel was established? 1948


In [21]:
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def remove_stop_words(text):
    tokens = word_tokenize(text)
    punct_or_stopwords = set([*stopwords.words('english'), *string.punctuation])
    return " ".join([token for token in tokens if token.lower() not in punct_or_stopwords])
def only_ents(text,doc):
    ans_idx = text.find('?')+2
    filtered_ans = remove_stop_words(text[ans_idx:])
    print(filtered_ans)
    return any([filtered_ans.strip() == cur_ent.text for cur_ent in doc.ents])

text = "what year Israel was established? few years after 1948."
doc =nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
print(only_ents(text,doc))
displacy.render(doc, style='dep')
displacy.render(nlp('few years after 1948'), style='dep',jupyter=True)

Israel 10 16 GPE
1948 50 54 DATE
years 1948
False


In [22]:
spacy.explain('npadvmod')

'noun phrase as adverbial modifier'

In [18]:
displacy.render(nlp('few years after 1948'), style='dep')

/home/ubuntu/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [4]:
spacy.explain('pobj')

'object of preposition'

In [1]:
'''
try:
    import spacy
except:
    import spacy
try:
    from get_knowledge_graph import from_text_to_kb
except:
    from get_knowledge_graph import from_text_to_kb

if isinstance(text,list):
    kb = from_text_to_kb(" ".join(text), "", verbose=True)
else:
    kb = from_text_to_kb(text, "",span_length=128, verbose=True)
kb.print()
'''
filename = "network_1_napoleon.html"

import IPython
#save_network_html(kb, filename=filename)
#save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

In [80]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Using cached newspaper3k-0.2.8-py3-none-any.whl (211 kB)
  Using cached GoogleNews-1.6.6-py3-none-any.whl (8.0 kB)
  Using cached pyvis-0.3.1.tar.gz (748 kB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
     |████████████████████████████████| 7.1 MB 24.4 MB/s eta 0:00:01
  Using cached tinysegmenter-0.3.tar.gz (16 kB)
  Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)
  Using cached tldextract-3.4.0-py3-none-any.whl (93 kB)
  Using cached feedfinder2-0.0.4.tar.gz (3.3 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
  Using cached jieba3k-0.35.1.zip (7.4 MB)
  Using cached datepars

In [17]:
with open('/home/ubuntu/Questions_generation/Locating and Editing Factual Associations in GPT.json') as f:
    text = json.load(f)
print([s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in text['sections'].items() ])

['MIT CSAIL.', 'Northeastern University.', 'MIT CSAIL Yonatan Belinkov† Technion – IIT.', 'We analyze the storage and recall of factual associations in autoregressive transformer language models, finding evidence that these associations correspond to localized, directly-editable computations. We first develop a causal intervention for identifying neuron activations that are decisive in a model’s factual predictions. This reveals a distinct set of steps in middle-layer feed-forward modules that mediate factual predictions while processing subject tokens. To test our hypothesis that these computations correspond to factual association recall, we modify feedforward weights to update specific factual associations using Rank-One Model Editing (ROME). We find that ROME is effective on a standard zero-shot relation extraction (zsRE) model-editing task. We also evaluate ROME on a new dataset of difficult counterfactual assertions, on which it simultaneously maintains both specificity and gener

/home/ubuntu/.local/lib/python3.8/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/ubuntu/.local/lib/python3.8/site-packages/spacy/displacy/__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [8]:
['generate question:'+i for i in COMES_sections]

['generate question:COMET is a recently proposed trainable neuralbased evaluation metric developed to assess the quality of Machine Translation systems. In this paper, we explore the usage of COMET for evaluating Text Summarization systems – despite being trained on multilingual MT outputs, it performs remarkably well in monolingual settings, when predicting summarization output quality. We introduce a variant of the model – COMES – trained on the annotated summarization outputs that uses MT data for pre-training. We examine its performance on several datasets with human judgments collected for different notions of summary quality, covering several domains and languages.',
 'generate question: Since manual annotation for any generative task is costly and time consuming, automatic metrics are commonly used to measure progress during training and compare output from independent systems. Metrics Shared Task collocated with the WMT workshop since 2008 (Callison-Burch et al., 2008) advances

In [7]:
'''from MCQ import flanT5MCQ
from datetime import datetime
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
dir_path = '../outputs/'+datetime.now().strftime("%d_%m_%y")+'_'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open(JSON_path) as f:
  result = json.load(f)

sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
sections = sections[:sections_num_max]
print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]
min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)
'''
import pandas as pd
questions_df = pd.read_pickle('/home/ubuntu/Questions_generation/outputs/03_01_23_finance_course1/GQ+QA+GQ.pkl')

questions_used = 'new_question'
questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
q_sim_mat,ans_sim_mat = mcq.find_similarity(questions_df[questions_used].to_list(),
                                            answers = questions_df['selected_ans'].to_list())
filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                  q_sim_mat=q_sim_mat,ans_sim_mat=ans_sim_mat,
                                 similarity_thrs=0.75, n_thrs=20, return_index=True)
questions_df['use_question']=False
questions_df.loc[filter_idx,'use_question']=True
questions_df

,index,section_n,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,new_question,RQUGE,take
0,6,0.0,1.0,COMET is a recently proposed trainable neuralb...,What is the purpose of this paper?,0.053599,To evaluate text summarization systems using C...,Using COMET to evaluate text summarization sys...,An evaluation model for text summarization sys...,This paper introduces a variant of the COMET m...,to assess machine translation quality,Use COMET for text summarization evaluation,evaluate text summarization systems,To evaluate the quality of text summarization ...,to evaluate text summarization systems using c...,To evaluate text summarization systems using C...,What is the purpose of this paper?,4.803611,True
1,18,2.0,1.0,"COMET is a trained metric that, based on sema...",The COMET score is calculated by what method?,0.002099,Based on semantic similarities between the tra...,semantic similarities between the translated a...,The COMET score is calculated by a stack of fe...,pre-trained multilingual language model to ext...,stack of feed-forward layers,human perception of translation quality,pre-trained multilingual language model,semantic similarities between the translated a...,pre-trained multilingual language model to ext...,pre-trained multilingual language model to ext...,What does COMET use?,4.583428,True
2,7,1.0,1.0,Since manual annotation for any generative ta...,Annotation-based generative models: a new appr...,0.000009,"Using annotated datasets, we propose an automa...",In this paper we propose an annotated version ...,In this paper we propose an annotated version ...,COMES2 is a variant of the Metric Shared Task ...,an annotation-based generative model,Using annotated dataset to evaluate quality of...,Computational Evaluation of Summary Quality of...,Annotation-based generative models: a new appr...,in this paper we propose an annotated version ...,In this paper we propose an annotated version ...,How is the model evaluated?,4.370208,True
3,9,1.0,1.0,Since manual annotation for any generative ta...,Predicting summary evaluation metrics using an...,0.000261,In this paper we propose an automatic method f...,In this paper we propose an automatic method f...,"Using annotated MT data, we propose a variant ...",We propose a variant of the model – COMES2 – t...,Computational Modeling for Summary Evaluation,Predicting summary evaluation metrics using an...,Using annotated MT data to predict summary eva...,Predicting summary evaluation metrics using an...,in this paper we propose an automatic method f...,In this paper we propose an automatic method f...,How is the COMES2 model evaluated?,4.240835,True
4,14,1.0,1.0,Since manual annotation for any generative ta...,How to evaluate the quality of a generative mo...,0.007374,This paper presents an approach to evaluate th...,"Using annotated datasets, we propose COMES2 to...","In this paper, we propose COMES2 – a variant o...",COMES2 is a variant of the model that uses the...,CoMES2,the use of the COMET metrics,we introduce COMET2 and evaluate its performan...,COMET metric by Rei et al. (2020),comes2 is a variant of the model that uses the...,COMES2 is a variant of the model that uses the...,How is COMES2 evaluated?,4.126691,False
5,12,1.0,1.0,Since manual annotation for any generative ta...,Using annotated WMT data for pre-training and ...,0.000979,This paper presents an approach to automatical...,Using annotated WMT data for pre-training and ...,The COMET metric is a framework for evaluating...,COMES2 is a variant of the Metrics Shared Task...,Computational Evaluation of Summary Models usi...,Computational Evaluation of Summary Models,Using annotated WMT data for pre-training and ...,COMET2: Using annotated WMT data for pre-train...,comes2 is a variant of the metrics shared task...,COMES2 is a variant of the Metrics Shared Task...,How is COMES2 evaluated?,4.089975,

In [3]:
import pandas as pd
questions_df = pd.read_pickle('/home/ubuntu/Questions_generation/outputs/03_01_23_finance_course1/GQ+QA+GQ.pkl')
questions_df

,section_n,section_n_chunk,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,new_question
0,0,0.0,1.0,Economics 252 is a course for undergraduates....,Which course is being filmed for open yale? ec...,0.000339,Financial markets,Economic markets,A lecturer for the course economics 252 is bei...,A lecturer for the course economics 252 is bei...,Finance,Economic markets,Financial markets,Economics 252,a lecturer for the course economics 252 is bei...,A lecturer for the course economics 252 is bei...,Who is being filmed?
1,0,0.0,1.0,Economics 252 is a course for undergraduates....,When did yale university first offer this course?,0.001526,Three years ago. the first time this course ha...,It was filmed for open yale in 2008.,"In 2008, three years ago. the first time this ...",The first time this course has been filmed for...,2009,2007,In 2008,Three years ago,"in 2008, three years ago. the first time this ...",The first time this course has been filmed for...,When was the last time this course was filmed?
2,0,0.0,1.0,Economics 252 is a course for undergraduates....,Where will you find the course?,0.003963,You'll find it online as part of open yale.,You'll find it online as part of open yale. it...,You'll find it online as part of open yale. it...,The course will be eventually posted on the in...,Online,Open yale,Yale university,On the internet,you'll find it online as part of open yale. it...,You'll find it online as part of open yale. it...,Where can you take Economics 252?
3,0,0.0,1.0,Economics 252 is a course for undergraduates....,In what year was this lecture recorded for ope...,0.007434,2008,It was recorded for open yale in 2008,"2008, three years ago. the first time this cou...",This lecture was recorded for open yale in 2008,,2008,,,2008,2008,When was the first time the course was filmed?
4,0,0.0,1.0,Economics 252 is a course for undergraduates....,Who was filmed for open yale?,0.007655,A lecturer,A lecturer for economics 252,Economics 252 instructor,A lecturer for economics 252 at yale university.,Economists,Professor,Economics 252,The speaker,a lecturer for economics 252 at yale university.,A lecturer for economics 252 at yale university.,Who is he?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,5,5.2,1.0,We're seeing a change in bank regulation that ...,Lecture 13 focuses on what type of contracts?,0.006115,Derivative contracts,Forward and futures markets,Derivative contracts that inject a lot of comp...,Derivative contracts that inject a lot of comp...,Future,Forward,Future markets,Stock options,derivative contracts that inject a lot of comp...,Derivative contracts that inject a lot of comp...,What are stock options?
70,5,5.2,1.0,We're seeing a change in bank regulation that ...,Who manages portfolios?,0.006300,Students in lecture 17 are learning about prof...,You don't have to be a billionaire to manage p...,David swensen is a professional money manager....,David swensen is a professional money manager.,A billionaire,Professionals,Money managers,Professional money managers,david swensen is a professional money manager....,David swensen is a professional money manager....,What did David Swensen say?
71,5,5.2,1.0,We're seeing a change in bank regulation that ...,Which lecture is about forwards and futures ma...,0.036790,Lecture13,15,13,The first lecture is lecture 13.,,15,13,Lecture 13,13,13,What lecture is about forwards and futures mar...
72,5,5.2,1.0,We're seeing a change in bank regulation that ...,What is the last lecture about?,0.067513,Financial markets and the financial industry.,Financial markets and the financial industry,The last lecture is about the revolution in fi...,This is a course not about making money,Moral purpose,Finance,Financial ethics,A moral purpose,this is a course not about making money,This is a course not about making money,What is the last lectu

In [8]:
#!pip install pytextrank
#!pip install lmqg
from lmqg import TransformersQG
# initialize model
del model
model = TransformersQG(language='en', model='lmqg/t5-large-squad-qg-ae')
# paragraph to generate pairs of question and answer
#context = "William Turner was an English painter who specialised in watercolour landscapes. He is often known as William Turner of Oxford or just Turner of Oxford to distinguish him from his contemporary, J. M. W. Turner. Many of Turner's paintings depicted the countryside around Oxford. One of his best known pictures is a view of the city of Oxford from Hinksey Hill."
# model prediction
question_answer = model.generate_qa(sections[0])
# the output is a list of tuple (question, answer)
print(question_answer)

TypeError: __init__() got an unexpected keyword argument 'device'

> /tmp/ipykernel_39270/3981189669.py(6)<module>()
      4 # initialize model
      5 del model
----> 6 model = TransformersQG(language='en', model='lmqg/t5-large-squad-qg-ae',device='cpu')
      7 # paragraph to generate pairs of question and answer
      8 #context = "William Turner was an English painter who specialised in watercolour landscapes. He is often known as William Turner of Oxford or just Turner of Oxford to distinguish him from his contemporary, J. M. W. Turner. Many of Turner's paintings depicted the countryside around Oxford. One of his best known pictures is a view of the city of Oxford from Hinksey Hill."



In [11]:

filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                 mcq.find_similarity(questions_df[questions_used].to_list()),
                                 similarity_thrs=0.75, n_thrs=20, return_index=True)
filter_idx

[0, 2, 4, 5, 10, 13, 14, 15, 21, 22]

In [8]:
from negspacy.negation import Negex

nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe('sentencizer')
nlp.add_pipe(Negex(nlp))

TypeError: __init__() missing 5 required positional arguments: 'name', 'neg_termset', 'ent_types', 'extension_name', and 'chunk_prefix'

> /tmp/ipykernel_3955/103320592.py(5)<module>()
      1 from negspacy.negation import Negex
      2 
      3 nlp = spacy.load("en_core_sci_sm")
      4 nlp.add_pipe('sentencizer')
----> 5 nlp.add_pipe(Negex(nlp))



In [16]:
# Construction via add_pipe with default model
# Use 'textcat_multilabel' for multi-label classification
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import MultiLabel_TextCategorizer
#textcat = nlp.add_pipe("textcat")
labels_list = ['Physics',	'Chemistry'	,'Astronomic Physics',	'Psychology',	'Business',	'Mathematics',	'History',	'Sociology',	'Economics',	'Political Science',	'Geology'	,'Philosophy',	'Art',	'Materials Science',	'Geography',	'Environmental Science',	'Computer Science',	'Engineering',	'Biology'	,'Medical Science']

# Construction via add_pipe with custom model
config = {"model": {"@architectures": "my_textcat"}}
#parser = nlp.add_pipe("textcat", config=config)

# Construction from class
# Use 'MultiLabel_TextCategorizer' for multi-label classification
from spacy.pipeline import MultiLabel_TextCategorizer
textcat = MultiLabel_TextCategorizer(nlp.vocab, DEFAULT_MULTI_TEXTCAT_MODEL, threshold=0.5)
textcat.add_label('chemistry')
[textcat.add_label(i) for i in labels_list]
textcat.labels

AttributeError: 'dict' object has no attribute 'has_dim'

In [2]:
import spacy
from negspacy.negation import Negex
#!pip install spacy
#!python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

doc = nlp("She does not like Steve Jobs but likes Apple products.")
for e in doc.ents:
    print(e.text, e._.negex)

/home/ubuntu/.local/lib/python3.8/site-packages/spacy/util.py:885: UserWarning: [W094] Model 'en_core_web_sm' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.3,<3.5.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /home/ubuntu/.local/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg

In [12]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

     |████████████████████████████████| 13.7 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 671 kB 72.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.7 MB 75.4 MB/s eta 0:00:01
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.0.0-py3-none-any.whl size=13704312 sha256=9005cfa5fcba0510ddcb2d3635fcb34905ac67596c410e72d67b8cc361e6d053
  Stored in directory: /home/ubuntu/.cache/pip/wheels/8b/21/c1/257748af7399fdaf1b2afc39c92fb839c436f42e67b656ff7e
Successfully built en-core-web-sm
ERROR: scispacy 0.5.1 has requirement spacy<3.5.0,>=3.4.0, but you'll have spacy 3.0.9 which is incompatible.
ERROR: en-core-sci-sm 0.5.1 has requirement spacy<3.5.0,>=3.4.1, but you'll have spacy 3.0.9 which is incompatible.
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully

In [13]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Aren’t you coming? Doesn’t he understand? Are you not coming? Does he not understand? Which of the following did not occur? Does Jeff is a real person? Name something David hadn't reveal in his vacation?")
any([token.dep_=='neg' for token in doc])

True

In [46]:
abrv_dict = {'ROME': 'Rank-one Model Editing', 'TE': 'Total Effect', 'IE': 'Indirect Effect', 'ATE': 'Average Total Effect', 'AIE': 'Average Indirect Effect', 'FT': 'Fine-tuning', 'KE': 'Knowledge Editor', 'ES': 'Efficacy Score', 'EM': 'Efficacy Magnitude', 'PS': 'Paraphrase Scores', 'NS': 'Neighborhood Score', 'KN': 'Knowledge Neurons'}
import re
def rep(text):
    target =lambda abrv,long_form_abrv: f'{long_form_abrv}'
    #`only_first` resolve only the first abbreviation in the text (e.g. if the text includes the results section, and the abbreviation was define at the introduction, the method will resolve once the abbreviation in the text the model is exposed to)
    for abrv,long_form_abbr in abrv_dict.items():
        text = text.replace(f'{long_form_abbr} ({abrv})',abrv) #first the function abbreviate all occurrences
        text = text.replace(f'{long_form_abbr}({abrv})',abrv) 
        #text = text.replace(long_form_abbr,abrv) 
        text = re.sub(re.compile(r''+long_form_abbr, re.IGNORECASE),abrv , text)
        pattern = re.compile(r'(^|\s|\.)('+abrv+')(\W)', re.IGNORECASE)
        text = re.sub(pattern,r"\1"+target(abrv,long_form_abbr)+ r'\3' ,  text, count=1)
    return text.strip() #.replace(target(abrv,long_form_abbr)+' ',target(abrv,long_form_abbr))

print(rep('The Rome is the best'))
print(rep('The ROME is the best ROME'))
print(rep('The rome is the best The Rank-one Model Editing is the best in Rome'))
print(rep('The rome is the best The rank-one Model Editing is the best in Rome'))
print(rep('Rank-one Model Editing (ROME) method for factual knowledge storage'))
print(rep('Rank-one Model Editing (ROME). method for factual knowledge storage'))
print(rep('Rank-one Model Editing (ROME) method for factual knowledge storage'))
print(rep('ROMEgt method for factual knowledge storage'))

The Rank-one Model Editing is the best
The Rank-one Model Editing is the best ROME
The Rank-one Model Editing is the best The ROME is the best in Rome
The Rank-one Model Editing is the best The ROME is the best in Rome
Rank-one Model Editing method for factual knowledge storage
Rank-one Model Editing. method for factual knowledge storage
Rank-one Model Editing method for factual knowledge storage
ROMEgt method for factual knowledge storage


In [2]:
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
from MCQ import flanT5MCQ

generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)

In [7]:
import re
def is_similar(string, patterns):
    # compile the regex pattern to remove spaces and punctuation marks
    string = string.replace('_','')
    pattern = r'[^\w\s]'
    regex = re.compile(pattern)

    # remove spaces and punctuation marks from the input string and the patterns
    string = regex.sub('', string)
    patterns = [regex.sub('', p) for p in patterns]

    # check if the input string is exactly the same as one of the patterns
    return string.lower().strip() in patterns
answers_black_list =['we','they','the authors','authors','author','the author','you','you are','we are',
                                'the speaker','speaker','the lecturer','lecturer',
                                'he','he is','she','she is',
                                  'I','these','those','they are','we do','it','is is']
inputs = ['we','We','we...','They!','  Go','THEY  _ ']
[is_similar(ak,answers_black_list) for ak in inputs]

[True, True, True, True, False, False]

In [20]:
def text_slicer( text):
        print(len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]))
        if len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]) <= mcq.tokenizer.model_max_length :
            return [text]
        chunks = []
        chunk = []
        length = 0
        for sentence in nltk.tokenize.sent_tokenize(text):
            _len = len(mcq.tokenizer.tokenize(sentence))
            if length + _len <= mcq.tokenizer.model_max_length :
                length += _len
                chunk.append(sentence)
            elif not chunk:
                # Can a sentence be applicable for splitting on to chunks?
                chunks.append(sentence.strip())
                length = 0
            else:
                chunks.append(' '.join(chunk).strip())
                chunk = [sentence]
                length = _len
        if chunk:
            chunks.append(' '.join(chunk).strip())
        return chunks
sections_chunks = []
sections_n = []
for i,cur_section in enumerate(sections):
    cur_section_chunks = text_slicer(cur_section)
    n_chunks = len(cur_section_chunks)
    sections_chunks.extend(cur_section_chunks)
    if n_chunks==1:
        sections_n.append(float(i))
    else:
        sections_n.extend([i+0.1*k for k in range(n_chunks)])
print('='*100)
[text_slicer(cur_section) for cur_section in sections_chunks]

376
244
963
534
1167
664
376
244
512
452
473
62
510
485
174
489
176


[[" The crisis began with bubbles in the stock market, housing market, and also in the commodities market. It's a financial crisis that's bigger than any since the Great Depression of the 1930's. There's many different ways of thinking about a crisis like this. And I wanted to focus on one way that people think about it in terms of probability models. So, that's not the only way, it's not necessarily my favorite way. Excuse my cold. I didn't bring any water. I hope I make it through this lecture.  There was a pre-break around 2000 when the stock market collapsed around the world. But then they came back again after 2003 and they were on another boom, like a roller coaster ride. That's the narrative story. And then, what happened is, we see a bunch of institutional collapses. We saw bank failures in the U.S. and then, we saw international cooperation to prevent this from spreading like a disease. So, we had governments all over the world bailing out their banks and other companies. That

In [19]:
len(mcq.tokenizer(sections, return_tensors="pt")["input_ids"][0]) 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True